# WARC Processing with Spark and Python

There are several [full featured](https://github.com/archivesunleashed/aut) [options](https://github.com/helgeho/ArchiveSpark) for processing [WARC](https://en.wikipedia.org/wiki/Web_ARChive) data with Scala in Spark. Unfortunately if you aren't familiar with Scala these can seem like a little bit of a black box, where you aren't quite sure what's going on behind the scenes. This can be particularly problematic when understanding what is going on behind the scenes is critical for your research.

Fortunately, if you happen to know Python there is [PySpark](https://spark.apache.org/docs/0.9.0/python-programming-guide.html) which allows you to process your WARC data with the clearly documented [warcio](https://pypi.org/project/warcio/) library. The JVM isn't able to optimize Python code the same way it can Scala. But these performance differences really depend on what you are doing. You may be willing to trade off a bit of performance for an increase in readability and comprehension.

This notebook is a brief guide for processing WARC data using PySpark and warcio. Somewhat unhelpfully it kind of assumes you are passingly familiar with Python, Spark and WARC already, and that you have working [Python3](https://www.python.org/downloads/) and [Spark](https://spark.apache.org/) environments installed. You may notice that some of the notes refer specifically to the [XSEDE platform](https://www.xsede.org/). You can comfortably ignore those bits if they aren't relevant to you.

## Install


First you need to install [warcio](https://pypi.org/project/warcio/) and [findspark](https://github.com/minrk/findspark). If you are working in your own environment you can probably just:

    pip install warcio findspark

*XSEDE Users: Since you don't have root to install things globally, and Jupyter runs in the context of their Anaconda environment you'll need to:*

    ssh edsu@bridges.psc.edu
    module load anaconda
    /opt/packages/anaconda/anaconda3-5.1.0/bin/pip install --user warcio findspark
    
If you know the location of your Spark installation you can set the `SPARK_HOME` environment variable, but if not `findspark` may be able to find it anyway. XSEDE users need to set this explicitly convincing:

In [1]:
import os
#os.environ['SPARK_HOME'] = '/opt/packages/spark/latest'

## Setup

Now we are ready to start up Spark here in our Jupyter notebook! We'll get a Spark Context and a Spark Session which are used for interacting with Spark from Python.

In [2]:
import findspark
findspark.init()

import pyspark
sc = pyspark.SparkContext(appName="warc-analysis")

import pyspark.sql
sqlc = pyspark.sql.SparkSession(sc)

## Input Data

Now lets create a list of all the WARC files we want to process. You can adjust the pattern to match where the data is on your filesystem. You may need to change this up depending on where your WARC files are stored.

In [3]:
import glob

warc_files = glob.glob('/Users/edsu/Data/spn/*/*.warc.gz')
print("found {} files".format(len(warc_files)))
print("e.g. {}".format(warc_files[0]))

found 47 files
e.g. /Users/edsu/Data/spn/liveweb-20131025181033/live-20131025001158-00002-wwwb-app2.us.archive.org.warc.gz


## Extractor Functions 

The needs of research are *many*, and it's not possible to anticipate all the possible ways you might want to analyze your WARC records. In fact, researchers are often concerned with asking *novel* questions, that have never been asked before. So they may not be interested in your code that has already answered it.

However in the case of web archive data, it should be possible to have a simple framework for writing your own functions that analyze WARC records and use Spark to apply them right? An *extractor* function is a simple Python function that takes a `warcio.ArcWarcRecord` as an argument and returns data for processing in Spark. We will take a look at a simple *extractor* function in a moment. But before we do, how will we apply this function to all of our WARC data?

To do do this we introduce the extractor *decorator*, which is a bit of syntactic sugar you can use to annotate your own *extractor functions*, to indicate that they are a function that will be iterating through a set of WARC records in a set of WARC files, and will apply your custom *extractor* function to each record as it goes. All the heavy lifting here is done by warcio, which is parsing the records.

In [4]:
import warcio

def extractor(f):
    def new_f(warc_files):
        for warc_file in warc_files:
            with open(warc_file, 'rb') as stream:
                for record in warcio.ArchiveIterator(stream):
                    yield from f(record)
    return new_f

So `extractor` is a function, that takes a function as an argument, that returns a new function. If this sounds complicated that's because, well, it is.

But as complicated as it sounds [decorators](https://www.python.org/dev/peps/pep-0318/) are a standard part of Python for a reason--they cut down on code repetition, which can lead to bloat and bugs. Fortunately you won't be writing any more functions like the one above.

Instead we will be writing consise (or complicated if you want) *extractor functions* that look something like the one below, which returns the URL belonging to *WARC Response* records in our WARC data (yes there are [other types](https://iipc.github.io/warc-specifications/specifications/warc-format/warc-1.1/#warc-type-mandatory) of WARC records).

In [5]:
@extractor
def url(record):
    if record.rec_type == 'response':
        yield (record.rec_headers.get_header('WARC-Target-URI'), )

This one is pretty simple, but there are a few things to note about *extractor functions*:

* You need to *deocrate* them with `@extractor` just before the function definition. Yep, this is actually `extractor` which we just defined.
* Your function needs to take one argument, which will be the WARC record to examine.
* Your function needs to `yield` a `tuple` containing some piece or pieces of information from the WARC record that you would like to process with Spark.

The *extractor functions* need to return a tuple because the thing being returned may be more complicated than a single value. And the function needs to *yield* instead of simply *return* because there may be more than one of these items to to return from a given WARC Record. For example, consider a function that returns all the hyperlinks in a WARC Response. More on this in a bit.

## Analyze

And finally for the fun part you've been waiting for--running the analysis! We us `parallelize` to tell Spark to process all of our WARC files:

In [6]:
warcs = sc.parallelize(warc_files)

At this point `warcs` is a Spark [Resilient Distributed Dataset](https://www.tutorialspoint.com/apache_spark/apache_spark_rdd.htm) (RDD). We then use *mapPartititions* to apply our extractor function to each one and get the result as...yes, another RDD.

In [7]:
output = warcs.mapPartitions(url)

We can convert our RDD to a Spark Data Frame (giving it column headers to use), which for comfort we can even turn into a [Pandas](https://pandas.pydata.org/) dataframe. 

In [8]:
df = output.toDF(["url"]).toPandas()

Ahhh pandas....

In [9]:
df.url.value_counts()

http://edge.quantserve.com/quant.js                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    730
http://pagead2.googlesyndication.com/pagead/show_ads.js                                                                                                                                                                                                                                                                                                                      

More soon ...